## semantic segmentation

### load the vgg data

In [1]:
import tensorflow as tf
import helper
import project_tests as tests
import os

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [ ]:
# sess = tf.Session()
# # must be ["vgg16"], an array. 
# tf.saved_model.loader.load(sess, ["vgg16"], "./data/vgg16")
# graph = tf.get_default_graph()

In [ ]:
# vgg_input_tensor = graph.get_tensor_by_name("image_input:0")
# vgg_keep_prob_tensor = graph.get_tensor_by_name('keep_prob:0')
# vgg_layer3_out_tensor = graph.get_tensor_by_name('layer3_out:0')
# vgg_layer4_out_tensor = graph.get_tensor_by_name('layer4_out:0')
# vgg_layer7_out_tensor = graph.get_tensor_by_name('layer7_out:0')

![vgg_architecutre](./data/vgg_archi-1.png)

In [ ]:
# print(vgg_input_tensor,    # shape=(?, ?, ?, 3), the 3 channel
#       vgg_keep_prob_tensor, 
#       vgg_layer3_out_tensor,  # (?, ?, ?, 256), the output of layer3, 256 channel. 1x1 convo
#       vgg_layer4_out_tensor,  # (?, ?, ?, 512), layer4 output, 512 channel, 1x1 convo
#       vgg_layer7_out_tensor)  # (?, ?, ?, 4096), layer 7, 4096 channel, replace fc with conv1x1

In [ ]:
# print(image_shape, layer3_shape, layer4_shape, layer7_shape)
# [ 1 160 576 3]   
# [ 1 20 72 256]   # 20 = 160 / 2 / 2 / 2 = 20 (three max pool, downsample with stride 2)
# [ 1 10 36 512]   # 10 = 20 / 2 (one max pool)
# [ 1 5 18 4096]   # 5 = 10 / 2 (from conv5 to FC4096)

In [2]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'

    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    graph = tf.get_default_graph()

    vgg_input_layer = graph.get_tensor_by_name(vgg_input_tensor_name)
    vgg_keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    vgg_layer3_out = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    vgg_layer4_out = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    vgg_layer7_out = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)

    # return None, None, None, None, None
    return vgg_input_layer, vgg_keep_prob, vgg_layer3_out, vgg_layer4_out, vgg_layer7_out

## define the layers

In [ ]:
#  question: print the dimension of each layer;

In [ ]:
# num_classes = 2
# conv_1x1 = tf.layers.conv2d(vgg_layer7_out_tensor, num_classes, 1, padding="same")
# # tf.nn.conv2d() vs tf.layers.conv2d(), what's the difference? 

# output_1 = tf.layers.conv2d_transpose(conv_1x1, num_classes, 4, strides=(2, 2), 
#                                       padding="same")
# # the dimension is not the same
# # fuse_1 = tf.add(output_1, vgg_layer4_out_tensor)
# # to make it addable, needs to convert vgg_pooling layers to 1x1 convolution
# layer4_converted = tf.layers.conv2d(vgg_layer4_out_tensor, num_classes, 1, padding="same")
# fuse_1 = tf.add(output_1, layer4_converted)

# output_2 = tf.layers.conv2d_transpose(fuse_1, num_classes, 4, strides=(2,2), padding="same")
# # fuse_2 = tf.add(output_2, vgg_layer3_out_tensor)
# # convert the layer4 to 1x1 convolution
# layer4_converted = tf.layers.conv2d(vgg_layer4_out_tensor, num_classes, 1, padding="same")
# fuse_2 = tf.add(output_2, layer4_converted)

# output_3 = tf.layers.conv2d_transpose(fuse_2, num_classes, 16, strides=(8,8), padding="same")

In [3]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer3_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    # convert the vgg_layer7 to 1x1 convolution
    layer7_conv = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, padding="same")
    # upsample
    layer7_upsample = tf.layers.conv2d_transpose(layer7_conv, num_classes, 4, strides=(2,2))
#     add vgg layer4 pooling layer, before the addition, convert the dimension
    layer4_conv = tf.layers.conv2d(vgg_layer4_out, num_classes, 1, padding="same")
    fuse1 = tf.add(layer7_upsample, layer4_conv)
    fuse1_upsample = tf.layers.conv2d_transpose(fuse1, num_classes, 4, strides=(2,2), 
                                                padding="same")
    
#     add vgg layer3 pooling, 
    layer3_conv = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, padding="same")
    fuse2 = tf.add(fuse1_upsample, layer3_conv)
    
    nn_last_layer = tf.layers.conv2d_transpose(fuse2, num_classes, 16, strides=(8,8), 
                                               padding="same")
    
    return nn_last_layer

### define the optimizer

In [4]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    labels = tf.reshape(correct_label, (-1, num_classes))
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    trainer = optimizer.minimize(loss)
    return logits, trainer, loss
#     return None, None, None
# tests.test_optimize(optimize)

In [5]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    # TODO: Implement function
    sess.run(tf.global_variables_initializer())
    for i in range(epochs):
      for images, lables in get_batches_fn(batch_size):
        _, loss = sess.run([train_op, cross_entropy_loss], 
                           feed_dict={input_image: images, 
                                      correct_label: lables,
                                      keep_prob: keep_prob,   # is this the problem to cause code hangs
                                      learning_rate: learning_rate
                                     })
#     pass

In [6]:
def run():
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/
    
    epochs = 3
    batch_size = 10
    
    correct_label = tf.placeholder(tf.int32, [None, None, None, num_classes], 
                                   name='correct_label')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')

    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # TODO: Build NN using load_vgg, layers, and optimize function
        vgg_input, vgg_keep_prob, vgg_layer3, vgg_layer4, vgg_layer7 = load_vgg(sess, vgg_path)
        nn_last_layer = layers(vgg_layer3, vgg_layer4, vgg_layer7, num_classes)
        logits, train_op, cross_entropy_loss = optimize(nn_last_layer, correct_label, 
                                          learning_rate, num_classes)
        # TODO: Train NN using the train_nn function
        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, vgg_input,
             correct_label, vgg_keep_prob, learning_rate)
        
        # TODO: Save inference data using helper.save_inference_samples
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, vgg_keep_prob, vgg_input)

        # OPTIONAL: Apply the trained model to a video

In [7]:
run()

Tests Passed
INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'


TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, numpy ndarrays, or TensorHandles.